<a href="https://colab.research.google.com/github/jrslagle/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_t_tests_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

## Practice 1-Sample T-tests

One Sample t-tests determine whether or not a sample mean is statistically different from some known (or hypothesized) population mean. 

### 1) Load the Data
- Use the [automobile dataset](https://archive.ics.uci.edu/ml/datasets/Automobile)
- Fix the column headers
- Make sure NaNs are used to indicate missing values

Feel free to add code cells and text cells as needed throughout the assignment.

In [83]:
# Import and clean the Automobile Dataset at UCI

import pandas as pd
import numpy as np

# Extract a list of column names from the dataset's landing page
text = !curl https://archive.ics.uci.edu/ml/datasets/Automobile
matches = [ re.search('\. ([\w-]+):', line) for line in text ]
column_names = [ match.group(1) for match in matches if match ]

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
auto_data = pd.read_csv(data_url, names=column_names)

# Translate some words into numbers
door_dict = {'two':2.0, 'four':4.0, '?':np.nan}
auto_data['num-of-doors'] = auto_data['num-of-doors'].replace(door_dict).apply(float)
cyl_dict = {'two':2, 'three':3, 'four':4, 'five':5, 'six':6, 'eight':8, 'twelve':12}
auto_data['num-of-cylinders'] = auto_data['num-of-cylinders'].replace(cyl_dict).apply(int)
# list(auto_data['num-of-cylinders'].value_counts().index)

# Replace all questions with NaN and set those columns to floats
question_columns = [column for column in auto_data.columns 
                    if (auto_data[column] == '?').sum() > 0]
for column in question_columns:
  auto_data[column] = auto_data[column].replace('?',np.nan).apply(float)

auto_data.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,2.0,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,4,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,2.0,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,4,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,2.0,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,6,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,4.0,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,4,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,4.0,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,5,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


In [73]:
auto_data.dtypes

symboling              int64
normalized-losses    float64
make                  object
fuel-type             object
aspiration            object
num-of-doors         float64
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders       int64
engine-size            int64
fuel-system           object
bore                 float64
stroke               float64
compression-ratio    float64
horsepower           float64
peak-rpm             float64
city-mpg               int64
highway-mpg            int64
price                float64
dtype: object

In [ ]:
auto_data.isnull().sum()

symboling             0
normalized-losses    41
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  4
stroke                4
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
price                 4
dtype: int64

In [ ]:
N = 15
start = N * 4
auto_data[start:start+15]

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
60,0,115,mazda,gas,std,4.0,sedan,fwd,front,98.8,177.8,66.5,55.5,2410,ohc,4,122,2bbl,3.39,3.39,8.6,84,4800,26,32,8495
61,1,129,mazda,gas,std,2.0,hatchback,fwd,front,98.8,177.8,66.5,53.7,2385,ohc,4,122,2bbl,3.39,3.39,8.6,84,4800,26,32,10595
62,0,115,mazda,gas,std,4.0,sedan,fwd,front,98.8,177.8,66.5,55.5,2410,ohc,4,122,2bbl,3.39,3.39,8.6,84,4800,26,32,10245
63,0,NaN,mazda,diesel,std,NaN,sedan,fwd,front,98.8,177.8,66.5,55.5,2443,ohc,4,122,idi,3.39,3.39,22.7,64,4650,36,42,10795
64,0,115,mazda,gas,std,4.0,hatchback,fwd,front,98.8,177.8,66.5,55.5,2425,ohc,4,122,2bbl,3.39,3.39,8.6,84,4800,26,32,11245
65,0,118,mazda,gas,std,4.0,sedan,rwd,front,104.9,175.0,66.1,54.4,2670,ohc,4,140,mpfi,3.76,3.16,8.0,120,5000,19,27,18280
66,0,NaN,mazda,diesel,std,4.0,sedan,rwd,front,104.9,175.0,66.1,54.4,2700,ohc,4,134,idi,3.43,3.64,22.0,72,4200,31,39,18344
67,-1,93,mercedes-benz,diesel,turbo,4.0,sedan,rwd,front,110.0,190.9,70.3,56.5,3515,ohc,5,183,idi,3.58,3.64,21.5,123,4350,22,25,25552
68,-1,93,mercedes-benz,diesel,turbo,4.0,wagon,rwd,front,110.0,190.9,70.3,58.7,3750,ohc,5,183,idi,3.58,3.64,21.5,123,4350,22,25,28248
69,0,93,mercedes-benz,diesel,turbo,2.0,hardtop,rwd,front,106.7,187.5,70.3,54.9,3495,ohc,5,183,idi,3.58,3.64,21.5,123,4350,22,25,28176


### 2) Pretend that this dataset represents the cars at a used car lot in your local town. 

- Use df.sample() to pick a random sample of 10 cars. Note that because this sample is random we are going to set the `random_state` so that all of us in the class get the same random sample. Please set your random state to `30` when using `df.sample()`

Is your sample reflective of the population value in regards to highway-mpg? Find the mean for `highway-mpg` for the entire dataset and use a 1-sample t-test to compare your estaimated sample mean to the population mean. Can you say that they are different? 

In [74]:
from scipy import stats

auto_sample = auto_data.sample(n=10, random_state=30)
sample_mean = auto_sample['highway-mpg'].mean()
pop_mean = auto_data['highway-mpg'].mean()
test = stats.ttest_1samp(a=auto_sample['highway-mpg'], popmean=pop_mean)
print(test)
print('sample mean =',format(sample_mean,'.2f'))
print('population mean =',format(pop_mean,'.2f'))

Ttest_1sampResult(statistic=0.8912422391619923, pvalue=0.39600040771093326)
sample mean = 32.80
population mean = 30.75


It appears this random sampling of cars is roughly representative of the population as far as highway mpg goes. Since the p-value is 0.396, there is a good chance this sample has the same mean as the population.

The salesman says the cars he sells typically have a fuel efficiency of about 35 miles per gallon on the highway. You want to verify his claim but can only test 10 cars. Using your sample of 10, test his claim and report your results.

In [75]:
test = stats.ttest_1samp(a=auto_sample['highway-mpg'], popmean=35)
p = test.pvalue
message = '''Dear Mr. Salesman,
  I have driven 10 cars and measured their gas 
mileages. It turns out that the mean gas mileage is definitely below 35 
(p < 5x10^-16). However, based on my sampling of 10 cars, I got unlucky and
can't provide evidence of this (p = 0.36).'''
print(message)

Dear Mr. Salesman,
  I have driven 10 cars and measured their gas 
mileages. It turns out that the mean gas mileage is definitely below 35 
(p < 5x10^-16). However, based on my sampling of 10 cars, I got unlucky and
can't provide evidence of this (p = 0.36).


Imagine that you now have the capacity to test 100 cars on the lot. Using the same random state of `30`, take a sample of 100 cars. Run a t-test to verify the salesman's claim with your new larger sample. Do you reach the same conclusion as you did with the sample size of 10? 

In [76]:
better_sample = auto_data.sample(n=100, random_state=30)
sample_mean = better_sample['highway-mpg'].mean()
pop_mean = auto_data['highway-mpg'].mean()
test = stats.ttest_1samp(a=better_sample['highway-mpg'], popmean=35)
p = test.pvalue
print(test)
print('sample mean =',format(sample_mean,'.2f'))
print('population mean =',format(pop_mean,'.2f'))
print('Salesman\'s claim = 35.00')

Ttest_1sampResult(statistic=-6.5185242467032305, pvalue=3.003137417468754e-09)
sample mean = 30.64
population mean = 30.75
Salesman's claim = 35.00


In [77]:
message = '''Dear Mr. Salesman,
  I have now driven 100 cars and measured their gas mileages. Now I am very
confident that the average gas mileage is less than 35 (p = 3x10^-9).'''
print(message)

Dear Mr. Salesman,
  I have now driven 100 cars and measured their gas mileages. Now I am very
confident that the average gas mileage is less than 35 (p = 3x10^-9).


Why might these two t-tests using the same dataset lead to different conclusions about the salesman's claim?

- with greater sample size, comes greater confidence. In math terms, as n grows, the 1/sqrt(n) used to calculate standard error approaches zero.

## Practice 2-Sample T-tests

Two Sample t-tests determine whether or not two sample means are statistically different from each other. 

This portion of your assignment requires you to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). 


### 1) Load the data

The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

- Read the dataset in from UCI, you'll need to provide a list of column headers
- Encode "yes" votes as 1 and "no" votes as 0. (You can use `df.replace()` to swap out these values)
- Use dataframe filtering to split the dataframe into two new dataframes based on party. Hold all republicans in one dataframe and all democrats in the other. These will be our two different "samples."



In [104]:
import pandas as pd
import re

# Extract a list of column names from the dataset's landing page
text = !curl https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records
matches = [ re.search('\. ([\w -]+): 2', line) for line in text ]
column_names = [ match.group(1) for match in matches if match ]

# Create dataframe
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'
vote_data = pd.read_csv(data_url, names=column_names)

# Replace y's and n's with 1's and 0's
y_n_dict = { 'y':1, 'n':0, '?':np.NaN }
issues = vote_data.columns[1:]
vote_data[issues] = vote_data[issues].replace(y_n_dict)

vote_data

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
431,democrat,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
432,republican,0.0,NaN,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
433,republican,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.0,1.0


### 2) Perform two sample T-tests on different issues and report the results.

- Find an issue that democrats support more than republicans with p < 0.01 (significant at the 99% level).
- Find an issue that republicans support more than democrats with p < 0.01 (significant at the 99% level).
- Find an issue where the difference between republicans and democrats has p > 0.1 (Not significant at the 90% level - i.e. there may not be much of a difference between the two sample means)

Please for each test that you run state your null and alternative hypothesis and then write a conclusion reflecting on the null and alternative hypothesis.

Remember, that two-sample t-tests will only tell us if the two groups are *different* from one another. We'll have to look at their sample means directly or use the sign on the t-statistic to know which of the two sample means is larger. 

In [115]:
from scipy import stats

# Split the data
dems = vote_data[vote_data['Class Name'] == 'democrat']
reps = vote_data[vote_data['Class Name'] == 'republican']

reps

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,republican,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
430,republican,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
432,republican,0.0,NaN,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
433,republican,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.0,1.0


The tests themselves were moved to the Stretch section

In [ ]:
# pair all issues once
for i, issueA in enumerate(issues):
  for issueB in issues[i+1:]:
    print(issueA,'vs',issueB)

## Stretch Goals:

### 1) Use functions and some iterator (for loop, .apply(), list comprehension, etc.) to perform two sample t-tests on every issue in the dataset in an automated fashion.

In [124]:
# Figure out which parties support which issues with a significance of p < 0.01

for issue in issues:
  dem_stance = dems[issue].dropna()
  rep_stance = reps[issue].dropna()
  test = stats.ttest_ind(dem_stance, rep_stance)
  p = test.pvalue
  significant = p < 0.01
  dem_support = dem_stance.mean() > rep_stance.mean()

  # "Please for each test that you run state your null and alternative hypothesis
  # and then write a conclusion reflecting on the null and alternative hypothesis."
  title_issue = issue.replace('-',' ').title()
  print('As for',title_issue+':')
  print('  Null Hypothesis = Republicans and Democrats with the same approval rate.')
  print('  Alternative Hypothesis = Republicans and Democrats with a differing approval rate.')
  print('  For this issue, the p value =',format(p, '.3f'))

  if p > 0.1:
    print('  Democrats and Republicans were about the same on',title_issue)
  elif significant and dem_support:
    print('  Democrats showed more support for',title_issue)
  elif significant and not dem_support:
    print('  Republicans showed more support for',title_issue)
  print()

As for Handicapped Infants:
  Null Hypothesis = Republicans and Democrats with the same approval rate.
  Alternative Hypothesis = Republicans and Democrats with a differing approval rate.
  For this issue, the p value = 0.000
  Democrats showed more support for Handicapped Infants

As for Water Project Cost Sharing:
  Null Hypothesis = Republicans and Democrats with the same approval rate.
  Alternative Hypothesis = Republicans and Democrats with a differing approval rate.
  For this issue, the p value = 0.929
  Democrats and Republicans were about the same on Water Project Cost Sharing

As for Adoption Of The Budget Resolution:
  Null Hypothesis = Republicans and Democrats with the same approval rate.
  Alternative Hypothesis = Republicans and Democrats with a differing approval rate.
  For this issue, the p value = 0.000
  Democrats showed more support for Adoption Of The Budget Resolution

As for Physician Fee Freeze:
  Null Hypothesis = Republicans and Democrats with the same appro



### 2) Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

### Be sure to check your work using Scipy!





In [ ]:
### YOUR WORK HERE

### 3) Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Be sure to check your work using Scipy!

In [ ]:
### YOUR WORK HERE